In [35]:
!pip --q install sentence_transformers torch
!pip install ipynb
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
from dataextracter import *

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
#load cleaned corpus data
#df=pd.read_csv("Data/clean_data.csv")
df=pd.concat([extractData("Data/fall_2022_nlp.json"), 
                    extractData("Data/spring_2022_nlp.json"),
                  extractData("Data/image_spring_2022_nlp.json"),
                  extractData("Data/image_fall_2022_nlp.json")])

# Clean
df = data_cleaning(df)

#Load language model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed all documents
corpus_embeddings = embedder.encode(df['Sentence'], convert_to_tensor=True)

# Query sentences:
#call data cleaning....
queries = ['What kind of model is used in brill tagger ','why is logistic regression more robust to correlated features', 'How to tackle out of vocabulary']


/content/dataextracter.py:95: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 95 of the file /content/dataextracter.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  data["Sentence"] = data["Sentence"].apply(lambda x: BeautifulSoup(str(x)).get_text())


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [34]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop most similar posts:")
    posts_seen=set()
    for score, idx in zip(top_results[0], top_results[1]):
      #Skip, if this post is already recommended
      post=df.iloc[idx.item()]['Post']
      # if post not in posts_seen:
      #   posts_seen.add(post)
      print("\nPost: {}\nDocument: {}\nScore: {:.4f}".format(post,df.iloc[idx.item()]['Sentence'],score))





Query: What kind of model is used in brill tagger 

Top most similar posts:

Post: 506
Document: is brill tagger a discriminative model or generative model?.
Score: 0.7269

Post: 506
Document: neither. the model does not output probabilities of tags or sequences of tags.
Score: 0.4479

Post: 506
Document: not a pure rule-based model. it is a combination of a statistical tagger and a rule-based tagger that learns rules automatically from the data.
Score: 0.3389

Post: 528
Document: it is a tradeoff of runtime and accuracy. my best recommendation is to try it both ways: implement your model with and without smoothing on the emissions, measure runtime and accuracy for both options, and decide if the tradeoff is worthwhile. looking at the theory, smoothing the emission probabilities is expected to cause a substantial increase in runtime, because now you are calculating paths into every tag for every word, instead of throwing away most of the paths, it also has the potential for an impr